# Find steering function
## Circle experiments: 2018-03-09<br>
Erik Frisk (<erik.frisk@liu.se>)<br>
Department of Electrical Engineering<br>
Linköping University<br>
Sweden

## Imports

In [ ]:
import sys
if not ('../python' in sys.path):
    sys.path.append('../python')

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from calib_util import LoadLogDataPickle, BoxOff
from calib_util import QualisysSpeed, OdoSpeed, GetCalibrationSection
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split

datadir = '../cascar_logs/20180309/'

In [ ]:
from pylab import rcParams
#%matplotlib 
%matplotlib 
rcParams['figure.figsize']=[12, 8]

## Load data

Load data. Unfortunately, automatic time synchronization does not work on these datasets. Therefore, time compensations are chosen by visual inspection.

In [ ]:
dataSets = ['circle_50_n100.pickle', 'circle_50_n80.pickle', 'circle_50_n60.pickle',
            'circle_50_n40.pickle', 'circle_50_n20.pickle', 'circle_50_0.pickle',
            'circle_50_20.pickle', 'circle_50_40.pickle', 'circle_50_60.pickle',
            'circle_50_80.pickle', 'circle_50_100.pickle']
steer = [-100, -80, -60, -40, -20, 0, 20, 40, 60, 80, 100] 
deltaT = [-4.8, 0, -2.95, -2, -3.5, -3, -2.5, -1.75, -2.7, -2.9, 1.1]  # Hand synchronized data
data = {}
for s, filename, dt in zip(steer, dataSets, deltaT):
    data[s] = LoadLogDataPickle(datadir + filename, 
                                body='cascar', n_skip=1, calibrate_phi=True,
                                synchronize=False)
    data[s][0]['t'] = data[s][0]['t'] + dt

Plot one data set

In [ ]:
pos, odo, _ = data[20]
v_qualisys = QualisysSpeed(pos)
v_odo = OdoSpeed(odo)

plt.figure(10)
plt.clf()
plt.plot(pos['x'], pos['y'])
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('square')
BoxOff()

plt.figure(11)
plt.clf()
plt.plot(pos['t'], v_qualisys, 'b')
plt.plot(odo['t'], v_odo, 'r')
plt.legend(['v_pos', 'v_odo'])
plt.xlabel('t [s]')
plt.ylabel('speed [m/s]')
BoxOff()

# Examine steer characteristics

Define time intervals in datasets suitable for simple radius estimation, and then use the relation 
$$\delta=\tan(\frac{L}{R})^{-1}$$
to estimate the wheel angle. The wheel base L is measured directly on the vehicle.

In [ ]:
steer = np.array([-100, -80, -60, -40, 20, 40, 60, 80, 100])
time_interval = [[16, 26], [16, 35], [16, 35], [16, 50], [16, 50], 
                 [10, 30], [10, 30], [10, 25], [10, 25]]
R = []
for s, ti in zip(steer, time_interval):
    pos, odo_l, odo_r = data[s]
    seg, x0 = GetCalibrationSection(pos, odo_l, ti[0], ti[1])
    R.append(np.mean([
        np.max(seg['x']) - np.min(seg['x']), 
        np.max(seg['y']) - np.min(seg['y'])]))
R = np.array(R)
L = 28.0/100
delta = np.arctan2(L, R)*np.sign(steer)

In [ ]:
plt.figure(20, clear=True)
plt.plot(steer, R)
plt.plot(steer, R, 'ro')
plt.xlabel('Steer [%]')
plt.ylabel('Radius [m]')
BoxOff()

Estimate affine model

In [ ]:
A = np.hstack((np.ones((len(steer), 1)), steer.reshape((-1, 1))))
b = delta.reshape((-1, 1))
theta_steer, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
theta_steer = theta_steer.reshape(-1)

def SteerFunction(steer, theta):
    return (theta[0] + theta[1]*steer)

Plot steer - delta characteristics along with estimated affine function

In [ ]:
plt.figure(30)
plt.clf()
plt.plot(steer, delta*180/np.pi)
plt.plot(steer, SteerFunction(steer, theta_steer)*180/np.pi, 'k')
plt.legend(['data', 'model'])
plt.plot(steer, delta*180/np.pi, 'ro')
plt.xlabel('Steer [%]')
plt.ylabel('Wheel angle [deg]')
plt.title('Steer characteristics')
BoxOff()

In [ ]:
print(theta_steer[0])
print(theta_steer[1])

# Estimate wheel radius

In [ ]:
def distance(seg):
    return np.sum(np.sqrt(np.sum(np.diff(seg[['x', 'y']], axis=0)**2, axis=1)))

def r_estimate(seg, Ll, L, theta):
    n = len(seg)
    delta = SteerFunction(seg['steer'].values[0], theta)
    s = distance(seg)
    r = s*(1-Ll/L*np.tan(delta))/(n*2*np.pi/10)
    return r

r0 = 78.0/1000/2
L = 28.0/100
Ll = 21.0/100/2

In [ ]:
# Ll = 17.0/100 # Gives minimum variance of rhat, infeasible
Ll = 21.0/100/2
rhat = []
for s, ti in zip(steer, time_interval):
    pos, odo_l, odo_r = data[s]
    seg, x0 = GetCalibrationSection(pos, odo_l, ti[0], ti[1])
    rhat.append(r_estimate(seg, Ll, L, theta_steer))
plt.figure(30)
plt.clf()
plt.plot(steer, rhat)
plt.plot(steer, rhat, 'ro')
plt.xlabel('steer [%]')
plt.ylabel('r_hat [mm]')
BoxOff()
print('mu = %.2f, interval: [%.2f, %.2f], interval len: %.2f' % (
    np.mean(rhat)*1000, np.max(rhat)*1000, np.min(rhat)*1000, (np.max(rhat)-np.min(rhat))*1000))

# Test odometry on circles

In [ ]:
def Odometry(segment, x0, c0, c1, r, L, W):
    def SteerFunction(steer, c0, c1):
        return c0 + steer*c1
    
    odo_hat = np.zeros((len(segment), 3))
    odo_hat[0, :] = x0[0:3]

    for k in range(0, len(segment)-1):
        delta = SteerFunction(segment['steer'].values[k], c0, c1)
        ds = np.pi*2*r/10*1/(1-W/2/L*np.tan(delta))
        odo_hat[k+1, :] = odo_hat[k, :] + ds*np.array([np.cos(odo_hat[k, 2]),
                                                       np.sin(odo_hat[k, 2]),
                                                       1/L*np.tan(delta)])

    odo_hat = pd.DataFrame(
        np.hstack((segment['t'].values.reshape(-1, 1), odo_hat)),
        columns=['t', 'x', 'y', 'phi'])
    
    return odo_hat

In [ ]:
pos, odo_l, odo_r = data[60]
seg, x0 = GetCalibrationSection(pos, odo_l, time_interval[6][0], time_interval[6][0]+5)

In [ ]:
plt.figure(80, clear=True)
plt.plot( seg['x'], seg['y'])
plt.axis('square')

In [ ]:
c0 = 0.02840204640670601
c1 = 0.0033528640065687834
r = 40.5/1000
L = 26.75/100
W = 21.0/100
theta0 = [c0, c1, r, L, W]

odo_hat = Odometry(seg, x0, c0, c1, r, L, W)
plt.figure(97, clear=True)
plt.plot(seg['x'], seg['y'], 'b')
plt.plot(odo_hat['x'], odo_hat['y'], 'r')
plt.plot(x0[0], x0[1], 'kx')
plt.axis('square')

In [ ]:
plt.figure(98, clear=True)
plt.plot(odo_l['dt'], 'b', label='left')
plt.plot(odo_r['dt'], 'r', label='right')
plt.plot(odo_l['dt'], 'ko', label='')
plt.plot(odo_r['dt'], 'ko', label='')

#plt.plot(odo_l['t'], odo_l['dt'], 'b', label='left')
#plt.plot(odo_r['t'], odo_r['dt'], 'r', label='right')

plt.legend()
